In [1]:
from route_calc import *

In [2]:
locations = read_locations("../data/locations.csv")

In [3]:
city_map = read_routes("../data/routes.csv", locations=locations)

In [4]:
plot_map(city_map).write_html("map.html")

In [5]:
city_map.calculate_duration("Museum of Fine Arts Boston", "Bunker Hill Monument")

12.0

In [6]:
city_map.calculate_duration("Bunker Hill Monument", "Museum of Fine Arts Boston")

12.0

In [7]:
# Make all routes experience general rush hour traffic
rush_hour_map = simulate_traffic(
    map=city_map,
    min_delay=1,
    max_delay=3,
    distribution=1,
    blockages=0,
)
# Add in a few extreme events
rush_hour_map = simulate_traffic(
    map=rush_hour_map,
    min_delay=10,
    max_delay=30,
    distribution=0.1,
    blockages=1,
)
plot_map(rush_hour_map).write_html("rush_hour.html")

In [8]:
rush_hour_map.calculate_duration("Museum of Fine Arts Boston", "Bunker Hill Monument")

25.414184202618458